<a href="https://colab.research.google.com/github/leoxiang66/Knowledge-driven-spoken-dialogue/blob/main/%E5%AF%B9%E8%AF%9D%E6%B6%89%E5%8F%8A%E7%9A%84%E5%AE%9E%E4%BD%93%E5%88%86%E7%B1%BB-model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 6.8 MB/s 
     |████████████████████████████████| 365 kB 56.5 MB/s 
     |████████████████████████████████| 6.6 MB 58.7 MB/s 
     |████████████████████████████████| 101 kB 14.8 MB/s 
     |████████████████████████████████| 212 kB 54.5 MB/s 
     |████████████████████████████████| 115 kB 60.1 MB/s 
     |████████████████████████████████| 127 kB 55.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from datasets import load_dataset

dataset = load_dataset("Adapting/Knowledge-Driven-Dialogues")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/Adapting___csv/Adapting--Knowledge-Driven-Dialogues-63071615aa101b1d/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
len(set(dataset['train']['knowledge_index']))

2503

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd 
%cd /content/drive/MyDrive/Downloads
%ls

/root
/content/drive/MyDrive/Downloads
knowledge_data.csv


In [ ]:
import pandas as pd
kg_data = pd.read_csv('knowledge_data.csv')
kg_data

,Unnamed: 0,entity,knowledge
0,0,调王生,[ett] 的时间是明朝。[ett] 的作者是袁凯。[ett] 的诗句是门外桃花落渐多，一双...
1,1,载伎重游王潭马砦岩,[ett] 的时间是明朝。[ett] 的作者是程嘉燧。[ett] 的诗句是疏松历落映层台，绿...
2,2,观军装十咏·马,[ett] 的时间是明朝。[ett] 的作者是高启。[ett] 的诗句是罗帕覆春风，连鞍赐有...
3,3,菩萨蛮 回文 中州乐府,[ett] 的时间是元朝。[ett] 的作者是孟宗献。[ett] 的诗句是睡惊秋近鸣虫砌。砌...
4,4,西江月·往事俄惊如梦,[ett] 的时间是元朝。[ett] 的作者是钱应庚。[ett] 的诗句是往事俄惊如梦，白头...
...,...,...,...
38049,38049,贝恩·血蹄,[ett] 的亲属(先祖)是血蹄长者。[ett] 的亲属(母亲)是塔玛拉。[ett] 的亲属...
38050,38050,怨恨回响,[ett] 的所在地是七星殿。[ett] 的种族是煞（元素生物）。[ett] 的等级是90。...
38051,38051,伊芙·费德里克森,[ett] 的势力是提瑞斯法议会。[ett] 的头衔是提瑞斯法议会的回忆。[ett] 的对联...
38052,38052,女伯爵莉亚德琳,[ett] 的亲属(养女)是萨兰蒂亚。[ett] 的亲属(导师/养父)是高阶牧师冯德洛尔。[...


In [ ]:
%cd 
%ls

/root


In [ ]:
num_labels = 38055

In [ ]:
from transformers import AutoModelForSequenceClassification
import torch
model_ckpt = "bert-base-chinese"
device = torch.device("cuda" if torch.cuda. is_available() else "cpu")

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

special_token_dict = {'additional_special_tokens': ['[usr]','[sys]']}
num_tokens = tokenizer.add_special_tokens(special_token_dict)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt,num_labels=num_labels).to(device)
model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Embedding(21130, 768)

In [ ]:
import ast
def preprocess_function(sample):
    chat_history = sample['chat_history']
    chat_history_list = ast.literal_eval(chat_history)
    chat_history = ''
    for i in range(0,len(chat_history_list)-1,2):
        chat_history += '[usr] '
        chat_history += chat_history_list[i]
        chat_history += ' [sys] '
        chat_history += chat_history_list[i+1]
        chat_history += ' '

    chat_history += f'[usr] {chat_history_list[-1]}'
    input = tokenizer(chat_history,truncation=True)
    
    if sample['knowledge_index'] == None:
        input['label'] = 38054
    else:
        input['label'] = sample['knowledge_index']
    
    return input

In [ ]:
tokenized_datasets = dataset.map(
    preprocess_function,
    remove_columns=dataset['train'].column_names
)

  0%|          | 0/25277 [00:00<?, ?ex/s]

  0%|          | 0/6563 [00:00<?, ?ex/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 25277
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 6563
    })
})

In [ ]:
tokenized_datasets['train'].features

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'label': Value(dtype='float64', id=None)}

In [ ]:
from datasets import ClassLabel
labels = ClassLabel(num_classes = num_labels)
tokenized_datasets = tokenized_datasets.cast_column("label", labels)

Casting the dataset:   0%|          | 0/3 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets['train'][:10]['label']

[21928, 21928, 21928, 21928, 21928, 21928, 8449, 15087, 15087, 38054]

In [ ]:
min(set(tokenized_datasets['train']['label']))

5739

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from datasets import load_metric

In [ ]:
GPU = torch.cuda.is_available()
torch.cuda.empty_cache()

training_args = TrainingArguments(
    output_dir="test-trainer", 
    num_train_epochs=10,
    save_total_limit=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    fp16=GPU,# speeds up training on modern GPUs.
    # eval_accumulation_steps = 10,
    )

In [ ]:


trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    # compute_metrics=compute_metrics,
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25277
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 15800


Epoch,Training Loss,Validation Loss
1,5.950600,7.282400


***** Running Evaluation *****
  Num examples = 6563
  Batch size = 16
Saving model checkpoint to test-trainer/checkpoint-1580
Configuration saved in test-trainer/checkpoint-1580/config.json
Model weights saved in test-trainer/checkpoint-1580/pytorch_model.bin
tokenizer config file saved in test-trainer/checkpoint-1580/tokenizer_config.json
Special tokens file saved in test-trainer/checkpoint-1580/special_tokens_map.json


# push

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()
